# Dense Networks · SP500 Next‑Day Direction (Binary Classification)

**Objective**: Predict tomorrow's SP500 move (up/down) using **lagged cross‑sectional stock returns** with a **Dense (MLP) network**.

You can run this notebook in **Google Colab**. It includes two ready experiments:
- **A:** Few stocks, many lags  
- **B:** Many stocks, few lags

🇬🇧 Note: This exercise belongs to the Dense Networks topic. Students compare how the number of lags and the number of stocks affect the model’s stability and performance.

🇷🇺 Подсказка: это практикум для темы *Dense Networks*. Студенты сравнивают влияние **числа лагов** и **числа акций** на устойчивость и качество модели.

## 0) Setup

In [ ]:
# If running in Colab:
# !pip -q install yfinance pandas numpy scikit-learn torch==2.4.1 matplotlib

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import TimeSeriesSplit

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(7)
np.random.seed(7)

print("Torch:", torch.__version__)

## 1) Data: choose one of the options

### Option 1 — Download with `yfinance` (easiest in Colab)
- Includes **SPY** for market proxy and a basket of large‑cap tickers.

In [ ]:
use_yfinance = True  # set to False if you upload your own CSV of returns

tickers_all = [
    "AAPL","MSFT","NVDA","AMZN","GOOG","META","BRK-B","TSLA","AVGO","LLY",
    "V","JPM","XOM","UNH","MA","HD","PG","COST","ADBE","NFLX",
    "PEP","CRM","KO","MRK","ABBV"
]
spy = "SPY"
start_date = "2012-01-01"
end_date = None  # or e.g. "2025-01-01"

if use_yfinance:
    try:
        import yfinance as yf
        data = yf.download([spy] + tickers_all, start=start_date, end=end_date, auto_adjust=True, progress=False)["Close"]
        data = data.dropna(how="all")
        data = data.asfreq("B").ffill()  # align to business days
        rets = data.pct_change().dropna()
        rets.columns = [c.replace(" ", "_").replace("-", "_") for c in rets.columns]
        spy_col = spy
        print("Data downloaded. Shape:", rets.shape)
    except Exception as e:
        print("yfinance failed:", e)
        use_yfinance = False
else:
    print("Set use_yfinance=True to auto-download, or upload your own CSV in the next cell.")

### Option 2 — Upload your own **returns** CSV
- Must contain **SPY** column (market proxy) and several stock columns
- Index must be a **date** (parseable)

In [ ]:
# If not using yfinance, upload your own daily RETURNS CSV with columns: SPY, AAPL, MSFT, ...
# Example:
# from google.colab import files
# uploaded = files.upload()
# rets = pd.read_csv(list(uploaded.keys())[0], index_col=0, parse_dates=True)
# spy_col = "SPY"

In [ ]:
assert 'rets' in globals(), "Please prepare `rets` (returns DataFrame) and `spy_col`."
assert spy_col in rets.columns, f"`{spy_col}` column required."
rets = rets.sort_index()
rets.head()

## 2) Target and Base Features

In [ ]:
# Target = tomorrow's SP500/market sign
y = (rets[spy_col].shift(-1) > 0).astype(int)

# Base cross-sectional features (exclude SPY to avoid leakage)
X_base = rets.drop(columns=[spy_col])
X_base.head(), y.head()

## 3) Helper: build lagged features

In [ ]:
def build_lagged_features(df: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    cols = {}
    for lag in range(1, n_lags + 1):
        lagged = df.shift(lag).add_suffix(f"_lag{lag}")
        cols[lag] = lagged
    X = pd.concat([cols[lag] for lag in cols], axis=1)
    return X

# Quick test
_ = build_lagged_features(X_base.iloc[:10], 3)

## 4) PyTorch dataset & model

In [ ]:
class TabDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32).reshape(-1, 1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class MLP(nn.Module):
    def __init__(self, in_dim: int, hidden=(64, 32), p_dropout=0.1):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers += [nn.Linear(last, h), nn.ReLU(), nn.Dropout(p_dropout)]
            last = h
        layers += [nn.Linear(last, 1)]
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)  # logits

In [ ]:
def train_model(X_train, y_train, X_val, y_val, epochs=50, batch_size=128, lr=1e-3, weight_decay=1e-4, hidden=(64,32)):

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s   = scaler.transform(X_val)

    train_ds = TabDataset(X_train_s, y_train)
    val_ds   = TabDataset(X_val_s,   y_val)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, drop_last=False)

    model = MLP(in_dim=X_train.shape[1], hidden=hidden, p_dropout=0.1)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_val = float("inf")
    best_state = None
    patience, patience_left = 8, 8

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(xb)
        train_loss /= len(train_ds)

        # validation
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            preds = []
            ys = []
            for xb, yb in val_loader:
                logits = model(xb)
                loss = criterion(logits, yb)
                val_loss += loss.item() * len(xb)
                preds.append(torch.sigmoid(logits).cpu().numpy())
                ys.append(yb.cpu().numpy())
            val_loss /= len(val_ds)
        if val_loss < best_val - 1e-5:
            best_val = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience_left = patience
        else:
            patience_left -= 1
            if patience_left <= 0:
                # early stop
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, scaler

In [ ]:
def evaluate_model(model, scaler, X, y, threshold=0.5, label="VAL"):
    Xs = scaler.transform(X).astype(np.float32)
    with torch.no_grad():
        logits = model(torch.from_numpy(Xs))
        probs = torch.sigmoid(logits).numpy().ravel()

    y_pred = (probs >= threshold).astype(int)
    acc  = accuracy_score(y, y_pred)
    bacc = balanced_accuracy_score(y, y_pred)
    try:
        roc  = roc_auc_score(y, probs)
    except ValueError:
        roc = np.nan
    cm = confusion_matrix(y, y_pred)

    print(f"[{label}] Accuracy={acc:.3f} | BalancedAcc={bacc:.3f} | ROC-AUC={roc:.3f}")
    print("Confusion matrix:\n", cm)
    return {"acc": acc, "bacc": bacc, "roc": roc, "cm": cm, "probs": probs, "pred": y_pred}

## 5) Time split: train / val / test

In [ ]:
def time_split_idx(n, train_frac=0.7, val_frac=0.15):
    train_end = int(n * train_frac)
    val_end   = int(n * (train_frac + val_frac))
    return slice(0, train_end), slice(train_end, val_end), slice(val_end, n)

## 6) Experiments A & B

In [ ]:
def run_experiment(name, stock_list, n_lags, hidden=(64,32), epochs=60):
    print(f"\n=== Experiment {name}: stocks={len(stock_list)}, lags={n_lags} ===")
    X_cs = X_base[stock_list]
    X = build_lagged_features(X_cs, n_lags)
    y_aligned = y.loc[X.index]

    # drop NA from shifts
    mask = X.notna().all(axis=1) & y_aligned.notna()
    X = X[mask]
    y_arr = y_aligned[mask].values

    n = len(X)
    tr, va, te = time_split_idx(n, 0.7, 0.15)
    X_train, y_train = X.iloc[tr].values, y_arr[tr]
    X_val,   y_val   = X.iloc[va].values, y_arr[va]
    X_test,  y_test  = X.iloc[te].values, y_arr[te]

    model, scaler = train_model(X_train, y_train, X_val, y_val, epochs=epochs, hidden=hidden)
    eval_val = evaluate_model(model, scaler, X_val, y_val, label=f"{name}-VAL")
    eval_te  = evaluate_model(model, scaler, X_test, y_test, label=f"{name}-TEST")

    return {"name": name, "eval_val": eval_val, "eval_test": eval_te}

# Define stock subsets
few_stocks  = ["AAPL","MSFT","NVDA","AMZN","GOOG"]
many_stocks = tickers_all if 'tickers_all' in globals() else list(X_base.columns)[:20]

# Run two contrasting experiments
res_A = run_experiment("A (few stocks, many lags)", stock_list=few_stocks,  n_lags=5, hidden=(64,32))
res_B = run_experiment("B (many stocks, few lags)", stock_list=many_stocks, n_lags=2, hidden=(128,64))

### 🧩 Concept Notes

#### **Bias–Variance Trade-off**
When we increase model complexity — for example, by adding more **lags**, **neurons**, or **layers** — the model can fit the training data more precisely but may lose its ability to generalize.  
This balance between **bias** (systematic error) and **variance** (sensitivity to noise) is central to all machine learning:

| Scenario | Bias | Variance | Description |
|-----------|------|-----------|--------------|
| Too simple model (few lags, small net) | high | low | underfitting — misses true patterns |
| Too complex model (many lags, deep net) | low | high | overfitting — memorizes noise |
| Balanced model | moderate | moderate | captures signal, ignores noise |

> 💡 In this exercise: adding too many lags can make the model memorize daily fluctuations instead of learning meaningful temporal structure.

---

#### **Cross-sectional Breadth**
Cross-sectional means “across many assets at the same time.”  
If **lags** add *temporal depth*, then **breadth** adds *market width* — the diversity of stocks used as simultaneous features.

| Scenario | What it means | Effect |
|-----------|----------------|---------|
| Few stocks, many lags | Focus on temporal behavior of each stock | Captures time-series structure but loses market context |
| Many stocks, few lags | Focus on market snapshot at each date | Captures cross-sectional structure but adds noise |

> 💡 Example: if the market index rises because only tech stocks surge while others fall, this is a **cross-sectional signal** — it can’t be seen from SP500 alone.


## 7) Discuss

EN:
- **Bias–Variance Trade-off**: Increasing the number of lags (time dimension) while keeping the same number of stocks expands the feature space — this raises the risk of overfitting, especially with shorter time series.

- **Cross-sectional Breadth**: Using more stocks with fewer lags adds cross-sectional information (market dispersion on a given day) but may introduce additional noise.

- **Metrics ≈ 0.50–0.55** on test data are typical for a clean setup without data leakage.
If ROC-AUC < 0.5, the model has learned a weak **anti-signal** — try inverting the target or adjusting feature definitions.


RU:
- **Bias–Variance trade‑off**: больше лагов (времени) при фиксированном числе акций увеличивает размер признакового пространства → риск переобучения при коротких рядах.  
- **Cross‑sectional breadth**: больше акций при малом числе лагов добавляет поперечную информацию (распределение рынка в конкретный день), но может усиливать шум.  
- **Метрики ≈ 0.50–0.55** на тесте — это нормально для «честной» постановки без утечки. Если ROC‑AUC < 0.5 — это **анти‑сигнал**; попробуйте инвертировать порог или признаки.

